In [ ]:
from datetime import timedelta, date
import os, time
import matplotlib.colors as colors
from netCDF4 import Dataset
from osgeo import gdal

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


def HexToRgb(hex_string):
    rgb = colors.hex2color(hex_string)
    r,g,b = tuple([int(255*x) for x in rgb])
    return (r,g,b)

def ProducePng(datestr):
    print datestr
    netcdf = 'NETCDF:"Map_%s.nc":pm25' % (datestr)
    src_ds = gdal.Open(netcdf)
    ulx, xres, xskew, uly, yskew, yres  = src_ds.GetGeoTransform()
    lrx = ulx + (src_ds.RasterXSize * xres)
    lry = uly + (src_ds.RasterYSize * yres)    
    cmd = "rm -f *.tif*"
    !$cmd
    cmd = 'GDAL_NETCDF_BOTTOMUP=NO gdal_translate -q -a_ullr %s %s %s %s -a_nodata -9999 -of GTiff  -b 1 %s %s.tif' % (ulx, lry, lrx, uly, netcdf, datestr)
    !$cmd
    cmd = 'gdalwarp -q -s_srs EPSG:4326 -t_srs EPSG:3857 -te -20037508.34 -20037508.34 20037508.34 20037508.34 -ts 1024 1024 -r cubicspline %s.tif %s-3857.tif' % (datestr, datestr)
    !$cmd
    cmd = 'gdaldem color-relief -q -alpha %s-3857.tif ramp.txt %s-color-3857.tif' % (datestr, datestr)
    !$cmd
    cmd = "convert %s-color-3857.tif %s.png" % (datestr, datestr)
    !$cmd    

In [ ]:
dates = []
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    datestr  = single_date.strftime("%Y%m%d")
    ncfilename = "Map_%s.nc" % (datestr)
    if os.path.exists(ncfilename):
        ProducePng(datestr)
        dates.append(single_date.strftime("%Y-%m-%d"))

In [ ]:
cmd = "ffmpeg -framerate 30 -pattern_type glob -i 'png/*.png' -c:v libx264 -s:v 512x512 -pix_fmt yuv420p -bf 0 -g 10 out.mp4"
!$cmd

In [ ]:
dates = []
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    datestr  = single_date.strftime("%Y%m%d")
    ncfilename = "Map_%s.nc" % (datestr)
    if os.path.exists(ncfilename):
        dates.append('"' + single_date.strftime("%Y-%m-%d") + '"')

In [ ]:
",".join(dates)